In [9]:
import csv
import random

def create_masked_csv(
    input_csv_path: str,
    separator: str = ":",
):
    """
    """

    with open(input_csv_path, "r", newline="", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)

    # Shuffle the rows randomly

    # Split the dataset into validation (20%) and training (80%)
    split_idx = len(rows) // 5
    all_rows = []
    for row in rows:
        all_rows.extend([row] * int(row['Redundancy']))
    rows = all_rows
    random.shuffle(rows)

    validation_rows = rows[:split_idx]
    training_rows = rows[split_idx:]

    fieldnames = ["id", "sequence"]

    # Write validation file
    with open("val.csv", "w", newline="", encoding="utf-8") as val_outfile:
        writer = csv.DictWriter(val_outfile, fieldnames=fieldnames)
        writer.writeheader()
        for row_id, row in enumerate(validation_rows):
            writer.writerow({
                "id": f"antibody_{row_id}",
                "sequence": f"{row['sequence_aa_heavy']}{separator}{row['sequence_aa_light']}"
            })

    # Write training file
    with open("train.csv", "w", newline="", encoding="utf-8") as train_outfile:
        writer = csv.DictWriter(train_outfile, fieldnames=fieldnames)
        writer.writeheader()
        for row_id, row in enumerate(training_rows, start=len(validation_rows)):
            writer.writerow({
                "id": f"antibody_{row_id}",
                "sequence": f"{row['sequence_aa_heavy']}{separator}{row['sequence_aa_light']}"
            })

# Run the function
create_masked_csv("comp_1.csv", separator=":")
